In [7]:
# Import modules
import Functional_Data_functions
from Functional_Data_functions import Smoothing_method,weights_init_normal,HyperParameters,Hyperparameter_Search,Hyperparameter_Test,Hyper_parameter_GridSearch,FCNN,Compile_class,Compile_train,from_torch_to_Datagrid

import inspect
import random
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn

from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
# matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import skfda as fda
from skfda import representation as representation
from skfda.exploratory.visualization import FPCAPlot
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
from tqdm import tqdm
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import gc
import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B


In [8]:
Phoneme=skfda.datasets.fetch_cran("phoneme",package_name="fda.usc")
phoneme=Phoneme['phoneme']
randices=np.random.randint(150,size=150)
# print(randices)
x_train,x_test,y_train,y_test=phoneme['learn'],phoneme['test'],phoneme['classlearn'],phoneme['classtest']
x_train.shape,x_test.shape,y_train.categories,y_test.categories
##Phonèmes enregistrés à classer dans 5 catégories
x_train,x_test,y_train,y_test=x_train[randices],x_test[randices],y_train[randices],y_test[randices]

c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")


In [16]:
x_train_tensor=torch.tensor(x_train.data_matrix).reshape(x_train.shape[0],1,150) .float()

x_test_tensor=torch.tensor(x_test.data_matrix).reshape(x_test.shape[0],1,150) .float()
y_train_tensor=torch.tensor(y_train.codes).unsqueeze(1).unsqueeze(2).long() 
y_test_tensor=torch.tensor(y_test.codes).unsqueeze(1).unsqueeze(2).long() 
X=torch.cat([x_train_tensor,x_test_tensor])
Y=torch.cat([y_train_tensor,y_test_tensor])
XY=torch.cat([X,Y],dim=2)
def from_torch_to_Datagrid(x):
    if isinstance(x,torch.Tensor):
        x_grid=fd(x[:,0,:].cpu(),grid_points=np.arange(x.shape[2]+1)[1:])
    elif isinstance(x,skfda.representation.grid.FDataGrid):
        x_grid=x
    else:
        raise ValueError("the NN argument must be either torch.tensor or skfda.representation.grid.FDataGrid")
    
    return x_grid



Choix de la méthode de smoothing on créer une instance de la classe Smoothing_method(), ici par exemple on choisit la base de Bspline et un smoothing au lieu de l'interpolation. On choisit la base avec les noeufs et l'ordre de spline, à noter que l'on peut également le choisir avec le nombre de fonctions de basis en mettant les knots_or_basis="basis" et n_basis=..., par défaut knots_or_basis="knots")
 

In [12]:
basis=B(knots=linspace(1,150,6),order=3)
##taille de la grille d'évaluation :
n=7500

Création des hyperparamètres: Le NN a trois couches et les paramètres de chaque couche sont modifiables dans l'instance de la classe HyperParameters(). 

In [27]:
negative_slope=0.17
class TSCNN(nn.Module):
    def __init__(self,):
        super(TSCNN, self).__init__()
    
        # Reste du code pour l'initialisation de la classe model
        self.convlayer1=nn.Sequential(
            nn.Conv1d(1,8,kernel_size=7,stride=95,padding=200,dilation=75),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(negative_slope),

            nn.Dropout(0.2),
            nn.MaxPool1d(kernel_size=3,stride=1,padding=0,dilation=1),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.2),
        )
        
        self.convlayer2=nn.Sequential(
            nn.Conv1d(8,512,kernel_size=5,stride=3,padding=2,dilation=2),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.2),
            
            nn.MaxPool1d(kernel_size=3,stride=2,padding=1,dilation=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.2),
        )
        
        self.convlayer3=nn.Sequential(

            nn.Conv1d(512,128,kernel_size=2,stride=1,padding=2,dilation=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope),
            
            nn.MaxPool1d(kernel_size=2,stride=1,padding=0,dilation=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope),
        )

        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(15*128,256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope),
            nn.Linear(256,5),
            
        )
        
    def Granulator(self,x):
        x_grid=from_torch_to_Datagrid(x=x)
        
        Recons_train=torch.zeros([x_grid.shape[0],1,n]).float() 
        eval_points=linspace(1,x_grid.grid_points[0].shape[0],n)
        coefs_torch=torch.from_numpy(x_grid.to_basis(basis=basis).coefficients).float() 
        basis_eval=basis(eval_points=eval_points,derivative=0)
        basis_fc = torch.from_numpy(basis_eval).float()         
        Recons_train=torch.matmul(coefs_torch,basis_fc[:,:,0])
        Recons_train=Recons_train.reshape(Recons_train.shape[0],1,n)
           
    
        return Recons_train.float() 



    def forward(self,x):
        Granulated_x_train=self.Granulator(x)
        Conv_out=self.convlayer1(Granulated_x_train)
        Conv_out2=self.convlayer2(Conv_out)
        Conv_out3=self.convlayer3(Conv_out2)
        Lin_out=self.fc_block(Conv_out3)
        return Lin_out.float().unsqueeze_(2).unsqueeze_(3)

In [33]:
#Création d'une instance de la classe:
TSC_model=TSCNN().apply(weights_init_normal)
Granul=TSC_model.Granulator(x_train)
Conv_out=TSC_model.convlayer1(Granul)
Conv_out2=TSC_model.convlayer2(Conv_out)
Conv_out3=TSC_model.convlayer3(Conv_out2)
n_conv_out1=Conv_out.shape[2]
n_conv_out2=Conv_out2.shape[2]
n_conv_out3=Conv_out3.shape[2]
##ON prend compte de la taille de la sortie 
# de la convolution pour actuliser la taille de la couche linéaire.
n_conv_out=n_conv_out3
print("taille de la sortie de convolution finale=",n_conv_out3)
print("Shape de la sortie du modèle=",TSC_model(X).shape)
# TSC_model.fc_block,TSC_model.n_conv_out*params.n_conv_in3


taille de la sortie de convolution finale= 15
Shape de la sortie du modèle= torch.Size([300, 5, 1, 1])


In [34]:
opt="SGD"
lr=0.00089
loss=nn.CrossEntropyLoss()
batch_size=100
betas = [0.5, 0.999]
if opt == "Adam":
    optimizer = optim.Adam(TSC_model.parameters(), lr=lr, betas=betas)
else:
    optimizer = optim.SGD(TSC_model.parameters(), lr=lr)
def train(n_epochs=1, module=TSC_model, optimizer=optimizer, loss=loss, batch_size=batch_size):
    training_acc=torch.tensor([0])
    testing_acc=torch.tensor([0])
    
    for epoch in tqdm(range(n_epochs)):
        train_loss = torch.tensor(0) .long()
        
        # Mélanger les données d'entraînement
        indices = list(range(len(x_train)))
        random.shuffle(indices)
        
        batch_index = 0  # Indice de batch
        
        for i in range(int(len(x_train) / batch_size)):
            # Obtenir les indices des données mélangées
            batch_indices = indices[batch_index:batch_index+batch_size]
            
            functions_train = x_train[batch_indices]
            labels_train = y_train_tensor[batch_indices]
            optimizer.zero_grad()
            output = module(functions_train)
            loss_value = loss(input=output, target=labels_train)
            
            loss_value.backward()
            optimizer.step()
            train_loss += loss_value.long()
            batch_index += batch_size  # Passer au prochain batch

        accuracy_training=((torch.sum(torch.argmax(TSC_model(x_train),dim=1)==y_train_tensor)/x_train_tensor.shape[0])*100).cpu().unsqueeze(0)
        accuracy=((torch.sum(torch.argmax(TSC_model(x_test_tensor),dim=1)==y_test_tensor)/x_test_tensor.shape[0])*100).cpu().unsqueeze(0)
        testing_acc=torch.cat([testing_acc,accuracy],dim=0)
        training_acc=torch.cat([training_acc,accuracy_training],dim=0)
        
    return training_acc,testing_acc

In [35]:
training_acc,testing_acc=train(n_epochs=20,module=TSC_model)
accuracy=((torch.sum(torch.argmax(TSC_model(x_test_tensor),dim=1)==y_test_tensor)/x_test_tensor.shape[0])*100)
print("Précision =",accuracy.detach().cpu().numpy(),"%")  
# accuracy.unsqueeze(0)

# training_acc=torch.cat([accuracy.cpu().unsqueeze(0),training_acc],dim=0)

100%|██████████| 20/20 [00:02<00:00,  6.75it/s]

Précision = 99.333336 %


In [38]:
torch.cuda.empty_cache()
gc.collect()

4478